## Zooniverse Analysis

In [ ]:
import shutil
import json
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
import seaborn as sns
from IPython.display import Image, display, HTML


In [ ]:
sns.set_context('poster')

In [ ]:
dfzoo = pd.read_csv('/epyc/users/ecbellm/ZTF_Boyajian/ztf-dippers-classifications.csv')

In [ ]:
len(dfzoo)

In [ ]:
dfzoo.head()

In [ ]:
wnew = dfzoo['created_at'].str.startswith('2021')

In [ ]:
np.sum(wnew)

In [ ]:
dfzoo = dfzoo[wnew]

this is one classification per row. but includes both v1 and v2 scanning

In [ ]:
dfzoo.iloc[0]['annotations']

we only have followup questions for the yes and maybe classificaitons, and some of the early yes/maybe classifications don't have followup at all

In [ ]:
dfzoo[['classification_id','user_name']].groupby('user_name').agg(len).sort_values(by='classification_id')

In [ ]:
def get_flat_outside_window(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[0]['value'])

def get_dip_both_bands(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[1]['value'] if (len(x) > 1) else None)

def get_asymmetric(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[2]['value'] if (len(x) > 1) else None)

def get_multiple_dips(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[3]['value'] if (len(x) > 1) else None)

def get_ps1_id(df):
    return dfzoo['subject_data'].apply(json.loads).apply(lambda x: list(x.values())[0]['ps1_id'])

In [ ]:
dfzoo['flat_outside_window'] = get_flat_outside_window(dfzoo)
dfzoo['dip_both_bands'] = get_dip_both_bands(dfzoo)
dfzoo['asymmetric'] = get_asymmetric(dfzoo)
dfzoo['multiple_dips'] = get_multiple_dips(dfzoo)
dfzoo['ps1_id'] = get_ps1_id(dfzoo)

In [ ]:
dfzoo['flat_outside_window'].value_counts()

In [ ]:
dfzoo.groupby(['ps1_id','flat_outside_window']).agg(len)['user_name'].value_counts()

In [ ]:
dfzoo['dip_both_bands'].value_counts()

In [ ]:
dfzoo['asymmetric'].value_counts()

In [ ]:
dfzoo['multiple_dips'].value_counts()

## Platinum sample: at least two votes w/ flat outside, multiple bands, asymmetric


In [ ]:
wplatinum = (dfzoo['flat_outside_window'] == 'Yes') & (dfzoo['dip_both_bands'] == 'Yes') & (dfzoo['asymmetric'] == 'Yes')

In [ ]:
np.sum(wplatinum)

In [ ]:
dfzoo.loc[wplatinum,'ps1_id'].value_counts()

In [ ]:
image_dir = '/astro/users/keatonb/ZTF_Boyajian/Candidate_plots/'

In [ ]:
for ps1_id, row in dfzoo.loc[wplatinum,'ps1_id'].value_counts().iteritems():
        print(ps1_id, row)
        display(Image(f'{image_dir}/{ps1_id}.png'))

## Gold sample: flat outside, could be single band, asymmetric 

In [ ]:
wgold = ((dfzoo['flat_outside_window'] == 'Yes') & 
        #((dfzoo['dip_both_bands'] == 'Yes') | 
         ((dfzoo['dip_both_bands'] == 'Not enough information to tell due to sampling.'))  
        & (dfzoo['asymmetric'] == 'Yes'))

In [ ]:
np.sum(wgold)

In [ ]:
for ps1_id, row in dfzoo.loc[wgold,'ps1_id'].value_counts().iteritems():
        print(ps1_id, row)
        display(Image(f'{image_dir}/{ps1_id}.png'))

### Lithium (or Palladium) sample: out-of-window variability that looks like dips

In [ ]:
wlithium = ((dfzoo['flat_outside_window'] == 'No, but the other excursions look like dips') & 
            (dfzoo['dip_both_bands'] == 'Yes') & (dfzoo['asymmetric'] == 'Yes'))

In [ ]:
np.sum(wlithium)

In [ ]:
for ps1_id, row in dfzoo.loc[wlithium,'ps1_id'].value_counts().iteritems():
        print(ps1_id, row)
        display(Image(f'{image_dir}/{ps1_id}.png'))

In [ ]:
# old v1 code put on ice for now

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(data=dfzoo,y='classification')
#plt.savefig(f'fig/{dataset}/classification_summary.png', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(data=dfzoo.loc[wyes],y='user_name')

In [ ]:
dfzoo.groupby('ps1_id').agg(len)['classification_id'].index[935]

In [ ]:
wmax_id = dfzoo['ps1_id'] == 172521268464107338
dfzoo.loc[wmax_id,['user_name','classification']]

In [ ]:
!ls

In [ ]:
image_dir = '/epyc/users/kyboone/ztf_boyajian/zooniverse/'

In [ ]:
display(Image(f'{image_dir}/172521268464107338.png'))

In [ ]:
wbest = wyes & wwell_sampled 
count_yes = dfzoo[['ps1_id','classification']].loc[wbest].groupby('ps1_id').agg(len)
wtwoyes = count_yes['classification'] > 1
best_ids = count_yes.loc[wtwoyes].index.values


In [ ]:
len(best_ids)

In [ ]:
print(best_ids)

In [ ]:
!mkdir -p gold_sample_figures

In [ ]:
!pwd

In [ ]:
for idi in best_ids:
    display(Image(f'{image_dir}/{idi}.png'))
    shutil.copyfile(f'{image_dir}/{idi}.png', f'gold_sample_figures/{idi}.png')

### Silver sample: at least one yes; number of yeses + maybes >= 2, no sampling check

In [ ]:
# at least one yes
count_yes = dfzoo[['ps1_id','classification']].loc[wyes].groupby('ps1_id').agg(len)
count_yes = count_yes.loc[count_yes['classification'] >= 1]

# any number of maybes
count_maybe = dfzoo[['ps1_id','classification']].loc[wmaybe].groupby('ps1_id').agg(len)

count_yes_and_maybe = count_yes.join(count_maybe,lsuffix='_yes',rsuffix='_maybe',how='outer')

In [ ]:
count_yes_and_maybe.sum(axis=1) 

In [ ]:
wokay = (count_yes_and_maybe.sum(axis=1) >= 2)
okay_ids = count_yes_and_maybe[wokay].index.values

In [ ]:
#remove the duplicated best ids
okay_ids = np.setdiff1d(okay_ids,best_ids)

In [ ]:
len(okay_ids)

In [ ]:
print(okay_ids)

In [ ]:
!mkdir -p silver_sample_figures

In [ ]:
for idi in okay_ids:
    display(Image(f'{image_dir}/{idi}.png'))
    shutil.copyfile(f'{image_dir}/{idi}.png', f'silver_sample_figures/{idi}.png')